In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
#from rnn_model import TRNNConfig, TextRNN
from tensorflow.contrib import learn
import os
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected

In [2]:
train = pd.read_csv("data/train_set.csv")
back_train = pd.read_csv('data/back_train.csv')
new_train = train.rename(columns={'class': 'article_class'}, inplace=False)
y_train = pd.get_dummies(new_train['article_class'])
back_y = pd.get_dummies(back_train['article_class']).values
y = y_train.values
y = np.concatenate((y, back_y), axis=0)


back_train = list(back_train.word_seg)
print(type(back_train),len(back_train))
print(type(back_train[0]),back_train[0])
new_train = list(new_train.word_seg)
print(type(new_train),len(new_train))
print(type(new_train[0]),new_train[0])
x_train = new_train+back_train

print(len(x_train))

<class 'list'> 49145
<class 'str'> 90540 816903 441039 816903 569138 816903 1034376 997489 382714 1236006 520477 132364 1207796 89613 1033299 834740 133940 816903 848000 386932 520477 31046 766772 748483 481786 701424 522428 764656 572782 266161 432549 457828 968083 397939 323159 1033823 925313 520477 848000 220238 701424 500399 133940 816903 1256303 520477 1243427 520477 487094 993110 477703 768219 133940 1034376 140644 98991 1207796 569876 1130139 1033823 257190 878073 585102 1105940 985047 834740 520477 506606 266784 781202 566120 768219 133940 816903 31046 422170 441513 703615 847492 520477 300241 54111 1278725 173393 652252 133940 1014945 1231069 1046814 1033823 637864 520477 1121767 614484 572782 266161 432549 457828 968083 397939 426716 487693 525987 323159 1033823 512031 853939 896267 133940 816903 426716 457828 1115586 816903 682718 816903 703473 122061 816903 266161 432549 526298 487337 1033823 788176 54111 973043 1033823 620853 874881 520477 806905 487337 520261 960230 10338

In [6]:
back_train[:5]

['90540 816903 441039 816903 569138 816903 1034376 997489 382714 1236006 520477 132364 1207796 89613 1033299 834740 133940 816903 848000 386932 520477 31046 766772 748483 481786 701424 522428 764656 572782 266161 432549 457828 968083 397939 323159 1033823 925313 520477 848000 220238 701424 500399 133940 816903 1256303 520477 1243427 520477 487094 993110 477703 768219 133940 1034376 140644 98991 1207796 569876 1130139 1033823 257190 878073 585102 1105940 985047 834740 520477 506606 266784 781202 566120 768219 133940 816903 31046 422170 441513 703615 847492 520477 300241 54111 1278725 173393 652252 133940 1014945 1231069 1046814 1033823 637864 520477 1121767 614484 572782 266161 432549 457828 968083 397939 426716 487693 525987 323159 1033823 512031 853939 896267 133940 816903 426716 457828 1115586 816903 682718 816903 703473 122061 816903 266161 432549 526298 487337 1033823 788176 54111 973043 1033823 620853 874881 520477 806905 487337 520261 960230 1033823 39741 138548 834740 806785 257

In [5]:
new_train[:5]

['816903 597526 520477 1179558 1033823 758724 632718 422098 520477 414956 625597 1203094 441513 596474 660569 995362 924085 1278762 1155376 1109510 823120 1275770 1203094 520477 831818 970779 1179558 758724 1033823 367432 995362 340401 599826 520477 907218 1080651 99188 188983 1238335 264536 834740 321830 327386 469755 295102 520477 106993 663234 670118 566120 960860 1033823 1032772 528558 1108771 643003 520477 1191581 560181 335962 1276143 995362 748896 776840 910464 520477 707234 235866 1113816 814750 636128 520477 1241273 756099 658505 477703 520477 653967 627858 147022 572824 995362 816903 566120 1255841 785043 1025743 197563 990423 816903 6562 1000483 520477 426552 876555 812998 1071452 965650 54111 1145850 1194044 885827 683784 1181092 1278762 520477 717686 527096 33842 858507 995362 473420 1245422 966562 430183 520477 488550 569876 113299 944337 796858 386932 520477 1161095 1188752 490535 635442 411707 881493 520477 183528 1112816 796858 454003 129186 960860 175827 520477 244780

In [3]:
vocab_processor = learn.preprocessing.VocabularyProcessor(1000,500)
vocab_processor.fit(back_train)

TypeError: expected string or bytes-like object

In [ ]:
test = pd.read_csv("data/test_set.csv")
x_test = list(test.word_seg)

x_text = x_train+x_test
#print(len(x_text))



# Build vocabulary
#max_document_length = max([len(x.split(" ")) for x in x_text])
vocab_processor = learn.preprocessing.VocabularyProcessor(1000,500)
#x = np.array(list(vocab_processor.fit_transform(x_text)))




vocab_dir = "vocab_dir/vocab"

#vocab_processor = learn.preprocessing.VocabularyProcessor.restore(vocab_dir)
vocab_processor.fit(x_train)
x = np.array(list(vocab_processor.transform(x_train)))
#print(len(x))
x_test = np.array(list(vocab_processor.transform(x_test)))

In [ ]:
pd.read_csv('baseline.csv')